### **Definir el directorio de trabajo con Google Colab**

Define el directorio de trabajo como la carpeta `data`.
Este directorio debe contener el archivo de datos original en formato comprimido `raw_data/Tomas.zip` y los diccionarios de datos `datatypes_dictionaries/dtypes_tomas` y `datatypes_dictionaries/dtypes_vars_toma`.

In [ ]:
import os
import sys
from google.colab import drive 
# Enlazar a la carpeta 'data'
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/ICBF/data')
sys.path.insert(0, '../scripts/0_utils')
!pwd

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/ICBF/data


### **Preprocesamiento del archivo por fragmentos**
Toma el archivo `Tomas.zip` (863 MB), originalmente codificados en formato `utf-16le`, del directorio `raw_data` y lo subdivide en `n` = 100 fragmentos, de ~60076 registros cada uno. Requiere el diccionario de datos del archivo `tomas_datatypes.json` con la estructura de datos predefinida.

#### **1. Lectura**
Lee todas las columnas columnas de cada fragmento, desde la fila que indica `particion[i]` hasta la fila que indica `particion[i + 1] - 1`, en un dataframe llamado `tom`.

#### **2. Preprocesamiento inicial**
Preprocesa este dataframe:
- Reemplaza la `'H '` de *hombre* por `'M'` de *masculino* y la `'M '` de *mujer* por `'F'` de *femenino* en la variable `Sexo`.
- Reemplaza comas por puntos en las columnas que según el diccionario de datos deben ser de valores flotantes.
- Corrige algunos errores ortográficos en las variables de `Estado`: Tilde en *desnutrición*, solo mayúscula inicial (formato oración).
- Reemplaza algunas fechas absurdas (>2800) encontradas para que puedan ser transformadas en tipo `datetime64`.
- Asigna el tipo adecuado a cada variable para mejorar el manejo de memoria.

#### **3. Segmentación en dataframes complementarios**

- **Datos por beneficiario**: Identificadores y primeras columnas, asociadas al beneficiario.

- **Datos de la Dirección de Primera Infancia**: Extrae los valores de las columnas asociadas a las tomas nutricionales `Toma1` a `Toma4` (realizadas por la Dirección de Primera Infancia) a otro dataframe llamado `tom_DPI`, con los datos apilados por toma.

- **Datos de la Dirección de Nutrición**: Genera una lista booleana `existen_tomas_N` que indica si existen o no datos no nulos en las columnas asociadas a las tomas nutricionales `Toma1N` a `Toma12N` (realizadas por la Dirección de Nutrición) y extrae estos valores a otro dataframe llamado `tom_DN`, con los datos apilados por toma.

- **Datos normalizados**: Los valores de las tomas nutricionales tomadas por ambas direcciones se apilan por tomas agregando un identificador por cada toma `IdToma` y se concatenan.

#### **Guardado**
Guarda los dataframes preprocesados en un formato binario comprimido `parquet`, que permite guardar la estructura de los tipos de datos, en el directorio `data_chunks`.
- Guarda el dataframe `tom_DN` completo en `TomasDNpre_#.parquet`.
- Guarda las primeras 98 columnas del dataframe `tom` (tomas nutricionales realizadas por la Dirección de Primera Infancia) en `TomasDPIpre_#.parquet`.

In [ ]:
# Librerias relevantes
import pandas as pd
import json
import numpy as np
import time
from utils import replace_log

In [ ]:
# Reemplazo de datos para asegurar su correcto procesamiento
# Reemplaza ',' por '.' en columnas de valores decimales
reemplazo_float = {',': '.'}
# Reemplaza 'M ' y 'f' por 'F', 'H ' por 'M' e 'I ' por 'I' en la columna 'Sexo'
reemplazo_sexo = {'M ': 'F',
                  'f':  'F',
                  'H ': 'M', 
                  'I ': 'I'}
# Corrige ortografía en las columnas de estado nutricional
reemplazo_estado = {'nutrici.n': 'nutrición',
                    ' Sobrepeso': ' sobrepeso',
                    ' Aguda': ' aguda',
                    ' Moderada': ' moderada'}
# Reemplaza las fechas absurdas (>2800) por su equivalente más probable                     
reemplazo_fecha = {'^2[89]': '20'}
 
# Abre los diccionarios con la estructura de datos definida
with open('datatypes_dictionaries/dtypes_tomas.json', 'r') as file:
  dtypes_tom = json.load(file)
with open('datatypes_dictionaries/dtypes_vars_toma.json', 'r') as file:
  dtypes_vars_toma = json.load(file)
 
# Nombres de las columnas
col_names = list(dtypes_tom.keys())
cols_var_toma = list(dtypes_vars_toma.keys())
# Columnas de identificadores: 'Registro', 'Vigencia', 'Id', 'IdBeneficiario'
cols_id = col_names[:4]
# Columnas propias del beneficiario
cols_beneficiario = col_names[:19]
# Columnas propias de cada toma nutricional
cols_tomas = cols_id + col_names[19:]
# Columnas propias de las tomas nutricionales de la Dirección de Primera Infancia
cols_DPI = cols_id + col_names[19:99]
# Columnas propias de las tomas nutricionales de la Dirección de Nutrición
cols_DN =  cols_id + col_names[99:]
 
# Columnas de variables tipo flotante
cols_float = [col for col in col_names if dtypes_tom[col] == 'float32']
# Columnas de la variable 'Toma': 'Toma1'... 'Toma12N'
cols_toma = [col for col in col_names if col[:4] == 'Toma']
# Columnas de variables tipo fecha
cols_fecha = [col for col in col_names if col[:5] == 'Fecha']
# Columnas de variables de estado nutricional: 'Estado...T1'... 'Estado...T12N'
# ('EstadoTallaEdad', 'EstadoPesoEdad',  'EstadoPesoTalla',  'EstadoIMC')
cols_estado = [col for col in col_names if col[:6] == 'Estado']

# Número total de registros, de fragmentos y número de registros en cada fragmento
n_registros = 6007562
n = 100
registros_fragmento = n_registros // n + 1
# Lista de números inicio y finalización de cada fragmento
# (El primer registro está en la fila 0, el último registro está en la fila 6007561)
particion = [registros_fragmento * i for i in range(n)] + [n_registros]

# Lista de fragmentos con tomas de la Dirección Nutricional
con_tomas_DN = list()
# Lista de reportes de cambios realizados
log = list()

In [ ]:
t0 = time.time()
# Para cada fragmento definido que empiece en particion[i] a particion[i + 1] - 1:
for i in range(n):
  # 1. Lee el fragmento completo
  # La tabla original no incluye la variable 'Registro'
  tom = pd.read_csv('raw_data/Tomas.zip', sep = '|', encoding = 'utf-16le', header = 0,
                    skiprows = range(particion[i]), nrows = particion[i + 1] - particion[i], 
                    names=col_names[1:])
  print(f'Fragmento {i + 1}/{n} leído: {time.time() - t0:.2f} s.')
  
  # 2. Preprocesamiento inicial del dataframe
  tom = tom.reset_index().rename(columns={'index': 'Registro'})
  # Asigna un número de registro (fila) adecuado, según la tabla original
  tom['Registro'] = pd.Series(range(particion[i], particion[i + 1]))
  # Realiza los reemplazos de valores descritos
  # Corrige ',' por '.' en columnas de valores decimales que no hayan sido leídas como tales
  for col in cols_float:
    if str(tom[col].dtypes)[:5] != 'float':
      tom[col] = tom[col].replace(reemplazo_float, regex = True)
  # Corrige las fechas absurdas en columnas de tipo fecha que no estén vacías
  for col in cols_fecha:
    if tom[col].notnull().sum() > 0:
      tom[col] = replace_log(tom, col, col_id='Registro', map=reemplazo_fecha, log=log)
  tom['Sexo'] = tom['Sexo'].replace(reemplazo_sexo, regex = False)
  tom[cols_estado] = tom[cols_estado].replace(reemplazo_estado, regex = True)
  # Asigna a las variables categóricas codificadas por enteros, el tipo de datos entero, 
  # para prevenir que se guarden como flotantes
  tom[cols_toma] = tom[cols_toma].astype('UInt16')
  tom['Vigencia'] = tom['Vigencia'].astype('UInt16')
  tom['CodigoPaisNacimiento'] = tom['CodigoPaisNacimiento'].astype('UInt16')
  # Asignar a cada variable según su tipo de acuerdo al diccionario de datos
  for col in col_names:
    tom[col] = tom[col].astype(dtypes_tom[col])
  print(f'Fragmento {i + 1}/{n} preprocesado: {time.time() - t0:.2f} s.')
  
  # 3. Segmentación en dataframes complementarios
  # Extraer los datos del beneficiario
  tom_beneficiario = tom[cols_beneficiario]
  # Extraer los registros que tienen alguna toma de cada dirección
  # (D. de Primera Infancia y D. de Nutrición)
  existen_tomas_DN = tom[cols_DN[4:]].notna().sum(axis='columns') > 0
  existen_tomas_DPI = tom[cols_DPI[4:]].notna().sum(axis='columns') > 0
  tom_DN = tom[cols_DN][existen_tomas_DN].reset_index(drop = True)
  # Descartar las variables de las tomas de la D. de Nutrición
  tom_DPI = tom[cols_DPI][existen_tomas_DPI].reset_index(drop = True)
  print(f'Fragmento {i + 1}/{n} extraídos {len(tom_DN)} registros de la D.N.:'+
        f'{time.time() - t0:.2f} s.')
  
  # 4. Normalización de las tomas nutricionales
  # Apilamiento (melt) guardando el identificador de la fila
  melted_DPI = tom_DPI.melt(id_vars = cols_id)
  # A cada [variable, valor] le asigna la toma según el sufijo de variable
  melted_DPI['toma'] = melted_DPI['variable'].str.extract(r'(\d+N*$)')
  # A cada [variable, valor] le asigna el nombre de variable apropiado (sin sufijos)
  melted_DPI['variable'] = melted_DPI['variable'].str.replace(r'T*\d+N*', '', regex=True)
  # Desapilamiento (pivot) para que los valores del mismo registro y toma se ubiquen en la misma fila
  pivot_DPI = melted_DPI.pivot(index=cols_id + ['toma'], columns='variable', values='value')
  # Remueve los valores que tienen un valor faltante en 'Toma' (toma nutricional faltante)
  pivot_DPI = pivot_DPI.dropna(subset=['Toma']).reset_index(drop=False)
  pivot_DPI['Toma'] = pivot_DPI['toma']
  # Asigna a todas las filas 'AntecedentePremadurez': '-' y 'Direccion': 'Primera Infancia'
  pivot_DPI['AntecedentePremadurez'] = pd.Series(['-'] * len(pivot_DPI))
  pivot_DPI['Direccion'] = pd.Series(['Primera Infancia'] * len(pivot_DPI))
  # Reordena las columnas
  pivot_DPI = pivot_DPI[cols_var_toma]
  if len(tom_DN) > 0:
    # Apilamiento (melt) guardando el identificador de la fila
    melted_DN = tom_DN.melt(id_vars = cols_id)
    # A cada [variable, valor] le asigna la toma según el sufijo de variable
    melted_DN['toma'] = melted_DN['variable'].str.extract(r'(\d+N*$)')
    # A cada [variable, valor] le asigna el nombre de variable apropiado (sin sufijos)
    melted_DN['variable'] = melted_DN['variable'].str.replace(r'T*\d+N*', '', regex=True)
    # Desapilamiento (pivot) para que los valores del mismo registro y toma se ubiquen en la misma fila
    pivot_DN = melted_DN.pivot(index=cols_id + ['toma'], columns='variable', values='value')
    # Remueve los valores que tienen un valor faltante en 'Toma' (toma nutricional faltante)
    pivot_DN = pivot_DN.dropna(subset=['Toma']).reset_index(drop=False)
    pivot_DN['Toma'] = pivot_DN['toma']
    # Asigna a todas las filas 'Direccion': 'Nutrición'
    pivot_DN['Direccion'] = pd.Series(['Nutrición'] * len(pivot_DN))
    # Reordena las columnas
    pivot_DN = pivot_DN[cols_var_toma]
    # Concatena la información de las tomas de ambas direcciones
    tom_melted = pivot_DPI.append(pivot_DN)
  else:
    tom_melted = pivot_DPI
  # Asigna cada columna al tipo de dato que le corresponde
  for col in tom_melted.columns:
    tom_melted[col] = tom_melted[col].astype(dtypes_vars_toma[col])
  print(f'Fragmento {i + 1}/{n} normalizado: {time.time() - t0:.2f} s.')
  
  # 5. Guardado de archivos
  # Guardar los dataframes preprocesado en un archivo .parquet
  tom_beneficiario.to_parquet(f'data_chunks/tomas_beneficiario/tomas_beneficiario_{i + 1}.parquet')
  tom_melted.to_parquet(f'data_chunks/tomas_normalizado/tomas_normalizado_{i + 1}.parquet')
  tom_DPI.to_parquet(f'data_chunks/direccion_primera_infancia/tomas_DPI_{i + 1}.parquet')
  # Guarda el dataframe con tomas de la Direccion de Nutrición solo si hay algún registro
  if (len(tom_DN) > 0):
    con_tomas_DN.append(i)
    tom_DN.to_parquet(f'data_chunks/direccion_nutricion/tomas_DN_{i + 1}.parquet')
  print(f'Fragmento {i + 1}/{n} guardado: {time.time() - t0:.2f} s.')  
  # Elimina los dataframes temporales
  del tom
  del tom_beneficiario
  del tom_DPI
  del tom_DN
  del tom_melted
print(f'Se extrajeron registros de {len(con_tomas_DN)}/{n} fragmentos: {time.time() - t0:.2f} s.')

Fragmento 1/100 leído: 5.09 s.
Fragmento 1/100 preprocesado: 22.51 s.
Fragmento 1/100 extraídos 0 registros de la D.N.: 22.69 s.
Fragmento 1/100 normalizado: 47.92 s.
Fragmento 1/100 guardado: 51.73 s.
Fragmento 2/100 leído: 55.03 s.
Fragmento 2/100 preprocesado: 72.04 s.
Fragmento 2/100 extraídos 0 registros de la D.N.: 72.21 s.
Fragmento 2/100 normalizado: 97.53 s.
Fragmento 2/100 guardado: 99.10 s.
Fragmento 3/100 leído: 103.42 s.
Fragmento 3/100 preprocesado: 120.68 s.
Fragmento 3/100 extraídos 0 registros de la D.N.: 120.86 s.
Fragmento 3/100 normalizado: 146.14 s.
Fragmento 3/100 guardado: 147.61 s.
Fragmento 4/100 leído: 152.78 s.
Fragmento 4/100 preprocesado: 169.96 s.
Fragmento 4/100 extraídos 0 registros de la D.N.: 170.14 s.
Fragmento 4/100 normalizado: 195.28 s.
Fragmento 4/100 guardado: 196.70 s.
Fragmento 5/100 leído: 202.76 s.
Fragmento 5/100 preprocesado: 220.16 s.
Fragmento 5/100 extraídos 0 registros de la D.N.: 220.33 s.
Fragmento 5/100 normalizado: 245.67 s.
Fragmen

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Fragmento 7/100 leído: 306.20 s.
Fragmento 7/100 preprocesado: 323.60 s.
Fragmento 7/100 extraídos 0 registros de la D.N.: 323.80 s.
Fragmento 7/100 normalizado: 348.97 s.
Fragmento 7/100 guardado: 350.30 s.


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3,13,14,98,100,102,103,104,105,106,107,108,109,110,111,112,113,115,116,118,120,121,123,124,125,126,127,128,129,130,131,132,133,134,136,137,140,141,143,144,147,151,152,153,154,156,157) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Fragmento 8/100 leído: 359.04 s.
Fragmento 8/100 preprocesado: 377.14 s.
Fragmento 8/100 extraídos 19 registros de la D.N.: 377.32 s.
Fragmento 8/100 normalizado: 401.93 s.
Fragmento 8/100 guardado: 404.48 s.


...

Fragmento 85/100 leído: 7650.51 s.
Fragmento 85/100 preprocesado: 7670.91 s.
Fragmento 85/100 extraídos 712 registros de la D.N.: 7671.10 s.
Fragmento 85/100 normalizado: 7697.62 s.
Fragmento 85/100 guardado: 7699.68 s.


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (103,104,105,109,124,125,126,130,144,145,146,150,164,165,166,170,183,184,185,189,200,201,203,204,205,206,207,208,209,210,211,212,213,215,216,218,220,221,223,224,225,226,227,228,229,230,231,232,233,235,236,238,240,241,243,244,245,246,247,248,249,250,251,252,253,255,256,258,260,261,263,264,265,266,267,268,269,270,271,272,273,275,276,278,280,281,283,286,287,288,290,291,292,293,295,296,298,300,301,303,306,307,308,310,311,312,313,315,316,318) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Fragmento 86/100 leído: 7784.05 s.
Fragmento 86/100 preprocesado: 7805.17 s.
Fragmento 86/100 extraídos 504 registros de la D.N.: 7805.36 s.
Fragmento 86/100 normalizado: 7832.08 s.
Fragmento 86/100 guardado: 7834.33 s.


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3,98,100,102,103,104,105,106,107,108,109,110,111,112,113,115,116,118,120,121,123,124,125,126,127,128,129,130,131,132,133,134,136,137,140,141,143,144,145,146,147,148,149,150,151,152,153,154,156,157,160,161,163,164,165,166,167,168,169,170,171,172,173,174,176,177,180,181,183,184,185,186,187,188,189,190,191,192,193,195,196,198,200,201,203,204,205,206,207,208,209,210,211,212,213,215,216,218,220,221,223,224,225,226,227,228,229,230,231,232,233,235,236,238,240,241,243,244,245,246,247,248,249,250,251,252,253,255,256,258,260,261,263,264,265,266,267,268,269,270,271,272,273,275,276,278,280,281,283,284,286,287,288,289,290,291,292,293,295,296,298) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Fragmento 87/100 leído: 7918.66 s.
Fragmento 87/100 preprocesado: 7939.64 s.
Fragmento 87/100 extraídos 157 registros de la D.N.: 7939.82 s.
Fragmento 87/100 normalizado: 7965.93 s.
Fragmento 87/100 guardado: 7967.96 s.
Fragmento 88/100 leído: 8053.37 s.
Fragmento 88/100 preprocesado: 8066.54 s.
Fragmento 88/100 extraídos 0 registros de la D.N.: 8066.71 s.
Fragmento 88/100 normalizado: 8092.64 s.
Fragmento 88/100 guardado: 8094.12 s.
Fragmento 89/100 leído: 8180.31 s.
Fragmento 89/100 preprocesado: 8193.16 s.
Fragmento 89/100 extraídos 0 registros de la D.N.: 8193.35 s.
Fragmento 89/100 normalizado: 8219.12 s.
Fragmento 89/100 guardado: 8220.73 s.
Fragmento 90/100 leído: 8308.38 s.
Fragmento 90/100 preprocesado: 8321.12 s.
Fragmento 90/100 extraídos 0 registros de la D.N.: 8321.31 s.
Fragmento 90/100 normalizado: 8346.12 s.
Fragmento 90/100 guardado: 8347.90 s.
Fragmento 91/100 leído: 8436.89 s.
Fragmento 91/100 preprocesado: 8449.66 s.
Fragmento 91/100 extraídos 0 registros de la D.N.

In [ ]:
t0 = time.time()
print('\nUnión de los fragmentos normalizados')
tom_normalizado = pd.DataFrame(columns = cols_var_toma)
# Toma cada archivo de tomas normalizado
for i in range(n):
  # Lee cada archivo en un dataframe y lo concatena al dataframe completo
  tom = pd.read_parquet(f'data_chunks/tomas_normalizado/tomas_normalizado_{i + 1}.parquet')
  tom_normalizado = tom_normalizado.append(tom)
  del tom
  if (i + 1) % 10 == 0:
    print(f'Partes {i + 1}/{n} unidas: {time.time() - t0:.2f} s.')
# Ordena las tomas por 'IdBeneficiario' y fecha de la toma
tom = tom.sort_values(['IdBeneficiario', 'FechaValoracionNutricional', 'Toma'])
# Genera un consecutivo de la toma nutricional 'IdToma'
tom = tom.reset_index(drop=True).reset_index().rename(columns={'index': 'IdToma'})
for col in tom_normalizado.columns:
  tom_normalizado[col] = tom_normalizado[col].astype(dtypes_vars_toma[col])
# Guarda el dataframe a un nuevo archivo .parquet
tom_normalizado.to_parquet('preprocessed_data/tomas_normalizado.parquet')
print(f'Archivo listo: {time.time() - t0:.2f} s.')


Unión de los fragmentos con muestras de la Dirección de Primera Infancia
Partes 10/100 unidas: 3.91 s.
Partes 20/100 unidas: 16.12 s.
Partes 30/100 unidas: 35.49 s.
Partes 40/100 unidas: 62.80 s.
Partes 50/100 unidas: 98.20 s.
Partes 60/100 unidas: 141.43 s.
Partes 70/100 unidas: 197.49 s.
Partes 80/100 unidas: 263.95 s.
Partes 90/100 unidas: 338.53 s.
Partes 100/100 unidas: 423.53 s.
Archivo listo: 463.37 s.


In [ ]:
t0 = time.time()
print('\nUnión de los fragmentos con datos del beneficiario')
tom_beneficiario = pd.DataFrame(columns = cols_beneficiario)
# Toma cada archivo de tomas normalizado
for i in range(n):
  # Lee cada archivo en un dataframe y lo concatena al dataframe completo
  tom = pd.read_parquet(f'data_chunks/tomas_beneficiario/tomas_beneficiario_{i + 1}.parquet')
  tom_beneficiario = tom_beneficiario.append(tom)
  del tom
  if (i + 1) % 10 ==0:
    print(f'Partes {i + 1}/{n} unidas: {time.time() - t0:.2f} s.')
# Evita la repetición de índices
tom_beneficiario = tom_beneficiario.reset_index(drop = True)
for col in tom_beneficiario.columns:
  tom_beneficiario[col] = tom_beneficiario[col].astype(dtypes_tom[col])
# Guarda el dataframe a un nuevo archivo .parquet
tom_beneficiario.to_parquet('preprocessed_data/tomas_beneficiario.parquet')
print(f'Archivo listo: {time.time() - t0:.2f} s.')


Unión de los fragmentos con datos del beneficiario
Partes 10/100 unidas: 1.33 s.
Partes 20/100 unidas: 4.28 s.
Partes 30/100 unidas: 9.30 s.
Partes 40/100 unidas: 15.52 s.
Partes 50/100 unidas: 22.78 s.
Partes 60/100 unidas: 31.58 s.
Partes 70/100 unidas: 41.69 s.
Partes 80/100 unidas: 53.21 s.
Partes 90/100 unidas: 66.11 s.
Partes 100/100 unidas: 80.43 s.
Archivo listo: 89.09 s.


### **Unión de los fragmentos homólogos**
Une los `n` = 100 fragmentos preprocesados `TomasDPIpre_#.parquet`, `TomasDNpre_#.parquet`, `Tomas_beneficiario_#.parquet` y `tomas_normalizado_#.parquet` de los subdirectorios del directorio `data_chunks` y en archivos únicos. Requiere el diccionario de datos del archivo `tomas_datatypes.json` con la estructura de datos definida.

####**Inicialización**
Define un dataframe vacío llamado `tom_DPI_complete`, asignándole los nombres de las columnas, según las primeras 98 llaves del diccionario de datos en el archivo `tomas_datatypes.json` del directorio `datatypes_dictionaries`.

####**Lectura**
Lee cada uno de los fragmentos `TomasDPIpre_#.parquet`
del directorio `data_chunks` en un dataframe llamado `tom`, asignándole los nombres de las columnas, según las primeras 98 llaves del diccionario de datos en el archivo `tomas_datatypes.json`.

#### **Unión**
Concatena el dataframe `tom` al dataframe `tom_DPI_complete` y lo borra.

#### **Preprocesamiento**
- Asigna el tipo adecuado a cada variable.
- Reinicia los índices para evitar repeticiones.

#### **Guardado**
Guarda el dataframe completo `tom_DPI_complete` en el archivo de formato binario comprimido `TomasDPIpre.parquet` (473 MB) en el directorio `preprocessed_data`.

In [ ]:
t0 = time.time()
print('\nUnión de los fragmentos con muestras de la Dirección de Primera Infancia')
tom_DPI_complete = pd.DataFrame(columns = cols_DPI)
# Toma cada archivo de tomas tomadas por la D. de Nutrición
for i in range(n):
  # Lee cada archivo en un dataframe y lo concatena al dataframe completo
  tom = pd.read_parquet(f'data_chunks/direccion_primera_infancia/tomas_DPI_{i + 1}.parquet')
  tom_DPI_complete = tom_DPI_complete.append(tom)
  del tom
  if (i + 1) % 10 == 0:
    print(f'Partes {i + 1}/{n} unidas: {time.time() - t0:.2f} s.')
# Evita la repetición de índices
tom_DPI_complete = tom_DPI_complete.reset_index(drop = True)
# Asignar a cada variable según su tipo de acuerdo al diccionario de datos
for col in tom_DPI_complete.columns:
  tom_DPI_complete[col] = tom_DPI_complete[col].astype(dtypes_tom[col])
# Guarda el dataframe a un nuevo archivo .parquet
tom_DPI_complete.to_parquet('preprocessed_data/tomas_DPI.parquet')
print(f'Archivo listo: {time.time() - t0:.2f} s.')


Unión de los fragmentos con muestras de la Dirección de Primera Infancia
Partes 10/100 unidas: 7.15 s.
Partes 20/100 unidas: 22.43 s.
Partes 30/100 unidas: 46.86 s.
Partes 40/100 unidas: 79.88 s.
Partes 50/100 unidas: 122.96 s.
Partes 60/100 unidas: 174.18 s.
Partes 70/100 unidas: 235.18 s.
Partes 80/100 unidas: 307.87 s.
Partes 90/100 unidas: 394.85 s.
Partes 100/100 unidas: 502.98 s.
Archivo listo: 538.61 s.


In [ ]:
import os.path
t0 = time.time()
print('\nUnión de los fragmentos con muestras de la Dirección de Nutrición')
tom_DN_complete = pd.DataFrame(columns = cols_DN)
# Toma cada archivo de tomas tomadas por la D. de Nutrición
for i in con_tomas_DN:
  if os.path.isfile(f'data_chunks/direccion_nutricion/tomas_DN_{i + 1}.parquet'):
    # Lee cada archivo en un dataframe y lo concatena al dataframe completo
    tom = pd.read_parquet(f'data_chunks/direccion_nutricion/tomas_DN_{i + 1}.parquet')
    tom_DN_complete = tom_DN_complete.append(tom)
    del tom
    if (i + 1) % 10 == 0:
      print(f'Partes {i + 1}/{n} unidas: {time.time() - t0:.2f} s.')
# Evita la repetición de índices
tom_DN_complete = tom_DN_complete.reset_index(drop = True)
for col in tom_DN_complete.columns:
  tom_DN_complete[col] = tom_DN_complete[col].astype(dtypes_tom[col])
# Guarda el dataframe a un nuevo archivo .parquet
tom_DN_complete.to_parquet('preprocessed_data/tomas_DN.parquet')
print(f'Archivo listo: {time.time() - t0:.2f} s.')


Unión de los fragmentos con muestras de la Dirección de Nutrición
Partes 10/100 unidas: 0.53 s.
Partes 30/100 unidas: 2.06 s.
Partes 40/100 unidas: 3.57 s.
Partes 50/100 unidas: 6.64 s.
Partes 60/100 unidas: 11.93 s.
Partes 70/100 unidas: 18.84 s.
Partes 80/100 unidas: 27.39 s.
Archivo listo: 35.59 s.


In [ ]:
t0 = time.time()
print('\nReporte de cambios')
# Escribe un archivo con los cambios generados
with open('log_files/log_tomas_1.json', 'w') as log_file:
  log_file.write('Preprocesamiento_Tomas:\n')
  for line in log:
    log_file.write('\t' + line + '\n')
print(f'Archivo listo: {time.time() - t0:.2f} s.')


Reporte de cambios
Archivo listo: 57.04 s.


In [ ]:
tomc = pd.read_parquet('data_chunks/tomas_normalizado/tomas_normalizado_94.parquet')
tom = pd.read_parquet('data_chunks/tomas_beneficiario/tomas_beneficiario_94.parquet')
tom.info(), tom.memory_usage().sum()/1e6 #10.8M 212 M

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60076 entries, 0 to 60075
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Registro                        60076 non-null  UInt32        
 1   Vigencia                        60076 non-null  int64         
 2   IdBeneficiario                  60076 non-null  UInt32        
 3   Id                              26655 non-null  UInt32        
 4   Tipo                            0 non-null      float64       
 5   FechaNacimiento                 60076 non-null  datetime64[ns]
 6   Sexo                            60076 non-null  category      
 7   CodigoPaisNacimiento            60076 non-null  int64         
 8   PaisNacimiento                  60076 non-null  category      
 9   PresentaDiscapacidad            60076 non-null  category      
 10  ZonaUbicacion                   60075 non-null  category      
 11  Gr

(None, 4.087592)

In [ ]:
tom_beneficiario = pd.read_parquet('preprocessed_data/tomas_beneficiario.parquet')
tom_beneficiario.info(), tom_beneficiario.memory_usage().sum()/1e6 #6M 349 MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6007561 entries, 0 to 6007560
Data columns (total 19 columns):
 #   Column                          Dtype         
---  ------                          -----         
 0   Registro                        UInt32        
 1   Vigencia                        int64         
 2   IdBeneficiario                  UInt32        
 3   Id                              UInt32        
 4   Tipo                            category      
 5   FechaNacimiento                 datetime64[ns]
 6   Sexo                            category      
 7   CodigoPaisNacimiento            float64       
 8   PaisNacimiento                  category      
 9   PresentaDiscapacidad            category      
 10  ZonaUbicacion                   category      
 11  GrupoEtnico                     category      
 12  MesesLactanciaMaternaExclusiva  UInt8         
 13  MesesLactanciaMaternaTotal      UInt8         
 14  PesoAlNacer                     float32       
 15

(None, 366.468909)

In [ ]:
tom_DPI = pd.read_parquet('preprocessed_data/tomas_DPI.parquet')
tom_DPI.info(), tom_DPI.memory_usage().sum()/1e6 #10.8M 212 M 78004

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5931324 entries, 0 to 5931323
Data columns (total 84 columns):
 #   Column                            Dtype         
---  ------                            -----         
 0   Registro                          UInt32        
 1   Vigencia                          int64         
 2   IdBeneficiario                    UInt32        
 3   Id                                UInt32        
 4   Toma1                             float64       
 5   ServicioT1                        category      
 6   FechaValoracionNutricionalT1      datetime64[ns]
 7   EdadMesesT1                       float32       
 8   FechaMedicionPerimetroBraquialT1  datetime64[ns]
 9   MedicionPerimetroBraquialT1       float32       
 10  PesoT1                            float32       
 11  TallaT1                           float32       
 12  ZScoreTallaEdadT1                 float32       
 13  ZScorePesoEdadT1                  float32       
 14  ZScorePesoTallaT1 

(None, 1892.102332)

In [ ]:
tom_DN = pd.read_parquet('preprocessed_data/tomas_DN.parquet')
tom_DN.info(), tom_DN.memory_usage().sum()/1e6 #10.8M 212 M

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78004 entries, 0 to 78003
Columns: 245 entries, Registro to AntecedentePremadurezT12N
dtypes: UInt32(3), UInt8(24), category(81), datetime64[ns](28), float32(96), float64(12), int64(1)
memory usage: 63.7 MB


(None, 66.789728)